In [66]:
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
import random
from feature_engine.imputation import MeanMedianImputer
from sklearn.cluster import KMeans
from pydriller import Repository
from numpy import mean

In [13]:
github_links = pd.read_csv('C:/Users/tanji/Desktop/SoftwareRemodularization/volatile_projects_complete_links_limit10_filtered.csv')
github_links = github_links.groupby(['project_name', 'project_link']).size().reset_index()
github_links = github_links.drop(columns=0)
github_links

,project_name,project_link
0,Alluxio,https://github.com/Alluxio/alluxio
1,Assertj-core,https://github.com/assertj/assertj-core
2,Atmosphere,https://github.com/Atmosphere/atmosphere
3,AxonFramework,https://github.com/AxonFramework/AxonFramework
4,Beam,https://github.com/apache/beam
5,Byte-buddy,https://github.com/raphw/byte-buddy
6,Camel,https://github.com/apache/camel
7,Cas,https://github.com/apereo/cas
8,Cassandra,https://github.com/apache/cassandra
9,Cxf,https://github.com/apache/cxf


In [8]:
main_df = None
directory = "Effort_Estimation_Results_3E_v2/"
for filename in os.listdir(directory):
    print(filename)
    if main_df is None:
        main_df = pd.read_csv(os.path.join(directory, filename))
        main_df['project_name'] = filename.replace('.csv', '')
    else:
        temp_df = pd.read_csv(os.path.join(directory, filename))
        temp_df['project_name'] = filename.replace('.csv', '')
        frames = [main_df, temp_df]
        main_df = pd.concat(frames)

Alluxio.csv
Assertj-core.csv
Atmosphere.csv
AxonFramework.csv
Beam.csv
Byte-buddy.csv
Camel.csv
Cas.csv
Cassandra.csv
Cxf.csv
Dbeaver.csv
Eclim.csv
Guacamole-client.csv
Hadoop.csv
Hawtio.csv
karaf.csv
Okhttp.csv
Openapi-generator.csv
Orientdb.csv
Redisson.csv
Rest-assured.csv
Spring-framework.csv
Spring-security.csv
Storm.csv
Tika.csv
Traccar.csv


In [9]:
main_df.head()

,kmean_label,time_taken_mean,time_taken_min,time_taken_max,time_taken_q10,time_taken_q20,time_taken_q25,time_taken_q30,time_taken_q40,time_taken_q50,...,actual_num_of_classes_touched_min,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std,commit_line_changed,refactoring_perc,refactoring_perc_time_taken,cocomoII_time_taken,geneticP_time_taken,softwareMaintenance_time_taken,fuzzy_time_taken
0,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,2,2,NaN,8,1.000000,15.00000,4.13952,2.477916,12.48,42.435515
1,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,1,1,NaN,346,0.225434,1.00000,179.03424,107.076009,2229.76,519.589347
2,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,1,2,0.707107,35,1.000000,1.00000,18.11040,10.840124,189.60,113.234791
3,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,1,13,4.018895,194,0.185567,3.71134,100.38336,60.060503,1232.64,353.641141
4,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,1,136,50.585661,1938,0.004128,1.00000,1002.79872,597.273303,12673.28,1634.048307


In [68]:
for item in main_df.columns:
    print(item)

kmean_label
time_taken_mean
time_taken_min
time_taken_max
time_taken_q10
time_taken_q20
time_taken_q25
time_taken_q30
time_taken_q40
time_taken_q50
time_taken_q60
time_taken_q70
time_taken_q75
time_taken_q80
time_taken_q90
sha
name
email
date
login
message
parent_sha
parent_date
time_taken
contains_refactoring
project_name
commit_compared_with
cbo_mean
cbo_min
cbo_max
cbo_std
wmc_mean
wmc_min
wmc_max
wmc_std
dit_mean
dit_min
dit_max
dit_std
rfc_mean
rfc_min
rfc_max
rfc_std
lcom_mean
lcom_min
lcom_max
lcom_std
totalMethods_mean
totalMethods_min
totalMethods_max
totalMethods_std
staticMethods_mean
staticMethods_min
staticMethods_max
staticMethods_std
publicMethods_mean
publicMethods_min
publicMethods_max
publicMethods_std
privateMethods_mean
privateMethods_min
privateMethods_max
privateMethods_std
protectedMethods_mean
protectedMethods_min
protectedMethods_max
protectedMethods_std
defaultMethods_mean
defaultMethods_min
defaultMethods_max
defaultMethods_std
abstractMethods_mean
abstractMe

In [10]:
main_df['project_name'].unique()

array(['Alluxio', 'Assertj-core', 'Atmosphere', 'AxonFramework', 'Beam',
       'Byte-buddy', 'Camel', 'Cas', 'Cassandra', 'Cxf', 'Dbeaver',
       'Eclim', 'Guacamole-client', 'Hadoop', 'Hawtio', 'karaf', 'Okhttp',
       'Openapi-generator', 'Orientdb', 'Redisson', 'Rest-assured',
       'Spring-framework', 'Spring-security', 'Storm', 'Tika', 'Traccar'],
      dtype=object)

In [14]:
main_df = main_df.merge(github_links, how='left', on='project_name')

In [16]:
main_df.tail()

,kmean_label,time_taken_mean,time_taken_min,time_taken_max,time_taken_q10,time_taken_q20,time_taken_q25,time_taken_q30,time_taken_q40,time_taken_q50,...,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std,commit_line_changed,refactoring_perc,refactoring_perc_time_taken,cocomoII_time_taken,geneticP_time_taken,softwareMaintenance_time_taken,fuzzy_time_taken,project_link
770,6,24.474860,1.0,167.0,1.0,2.0,4.0,5.0,10.0,16.0,...,3,0.866025,16,0.750000,18.750000,8.27904,4.955729,64.96,67.284406,https://github.com/traccar/traccar
771,6,24.474860,1.0,167.0,1.0,2.0,4.0,5.0,10.0,16.0,...,10,2.946898,1696,0.004127,1.000000,877.57824,523.020584,11085.76,1495.349884,https://github.com/traccar/traccar
772,6,24.474860,1.0,167.0,1.0,2.0,4.0,5.0,10.0,16.0,...,1,0.000000,535,0.725234,17.405607,276.83040,165.484354,3469.60,694.272395,https://github.com/traccar/traccar
773,7,27.645283,1.0,157.0,2.0,4.0,5.0,8.2,13.0,17.0,...,1,NaN,4,1.000000,93.000000,2.06976,1.238971,-13.76,26.763600,https://github.com/traccar/traccar
774,14,22.417476,1.0,144.0,1.0,4.0,6.0,8.0,12.0,16.0,...,3,NaN,63,1.000000,22.000000,32.59872,19.510808,373.28,167.392539,https://github.com/traccar/traccar


In [17]:
len(main_df)

775

In [19]:
from pydriller.metrics.process.change_set import ChangeSet
metric = ChangeSet(path_to_repo='https://github.com/Alluxio/alluxio',
                   from_commit='a4dc54f7dc0333da096aa779dbaa79060c90d1ad',
                   to_commit='1356c0b35fd753d1081665b2c8f6b25da62bf2e8')

maximum = metric.max()
average = metric.avg()
print('Maximum number of files committed together: {}'.format(maximum))
print('Average number of files committed together: {}'.format(average))

Maximum number of files committed together: 2
Average number of files committed together: 2


In [28]:
rm = Repository('https://github.com/Alluxio/alluxio', single='a4dc54f7dc0333da096aa779dbaa79060c90d1ad').traverse_commits()
for commit in rm:
    print(commit.dmm_unit_size)
    print(commit.dmm_unit_complexity)
    print(commit.dmm_unit_interfacing)

0.0
1.0
1.0


In [46]:
from pydriller.metrics.process.contributors_count import ContributorsCount
from numpy import mean
metric = ContributorsCount(path_to_repo='https://github.com/Alluxio/alluxio',
                           from_commit='a4dc54f7dc0333da096aa779dbaa79060c90d1ad',
                           to_commit='1356c0b35fd753d1081665b2c8f6b25da62bf2e8')
count = metric.count()
minor = metric.count_minor()
avg_count = mean(list(count.values()))
avg_count_minor = mean(list(minor.values()))

print('Number of contributors per file: {}'.format(count))
print('Number of "minor" contributors per file: {}'.format(minor))

print('Average number of contributors per file: {}'.format(avg_count))
print('Average number of "minor" contributors per file: {}'.format(avg_count_minor))

Number of contributors per file: {'shell\\src\\main\\java\\alluxio\\cli\\fs\\command\\LeaderCommand.java': 1, 'tests\\src\\test\\java\\alluxio\\client\\cli\\fs\\command\\LeaderCommandIntegrationTest.java': 1, '.github\\workflows\\java8_integration_tests_webui.yml': 1, '.github\\workflows\\java8_unit_tests.yml': 1}
Number of "minor" contributors per file: {'shell\\src\\main\\java\\alluxio\\cli\\fs\\command\\LeaderCommand.java': 0, 'tests\\src\\test\\java\\alluxio\\client\\cli\\fs\\command\\LeaderCommandIntegrationTest.java': 0, '.github\\workflows\\java8_integration_tests_webui.yml': 0, '.github\\workflows\\java8_unit_tests.yml': 0}
Average number of contributors per file: 1.0
Average number of "minor" contributors per file: 0.0


In [53]:
from pydriller.metrics.process.contributors_experience import ContributorsExperience
metric = ContributorsExperience(path_to_repo='https://github.com/Alluxio/alluxio',
                                from_commit='a4dc54f7dc0333da096aa779dbaa79060c90d1ad',
                                to_commit='1356c0b35fd753d1081665b2c8f6b25da62bf2e8')
files = metric.count()
avg_files_contributor = mean(list(files.values()))
print('Files: {}'.format(files))
print(f'Average percentage of contributors experience {avg_files_contributor}')

Files: {'shell\\src\\main\\java\\alluxio\\cli\\fs\\command\\LeaderCommand.java': 100.0, 'tests\\src\\test\\java\\alluxio\\client\\cli\\fs\\command\\LeaderCommandIntegrationTest.java': 100.0, '.github\\workflows\\java8_integration_tests_webui.yml': 100.0, '.github\\workflows\\java8_unit_tests.yml': 100.0}
Average percentage of contributors experience 100.0


In [59]:
from pydriller.metrics.process.hunks_count import HunksCount
metric = HunksCount(path_to_repo='https://github.com/Alluxio/alluxio',
                                from_commit='a4dc54f7dc0333da096aa779dbaa79060c90d1ad',
                                to_commit='1356c0b35fd753d1081665b2c8f6b25da62bf2e8')
files = metric.count()
avg_hunk = mean(list(files.values()))
print('Hunk: {}'.format(files))
print(f'Average Hunk {avg_hunk}')

Hunk: {'shell\\src\\main\\java\\alluxio\\cli\\fs\\command\\LeaderCommand.java': 3, 'tests\\src\\test\\java\\alluxio\\client\\cli\\fs\\command\\LeaderCommandIntegrationTest.java': 1, '.github\\workflows\\java8_integration_tests_webui.yml': 1, '.github\\workflows\\java8_unit_tests.yml': 1}
Average Hunk 1.5


In [73]:
def get_pydriller_process_metrics(row):
    
    path_to_repo = row['project_link']
    from_commit = row['sha']
    to_commit = row['parent_sha']
    
    from pydriller.metrics.process.change_set import ChangeSet
    metric = ChangeSet(path_to_repo=path_to_repo,
                   from_commit=from_commit,
                   to_commit=to_commit)

    row['maximum_files_committed_tgt'] = metric.max()
    #average_files_committed_tgt = metric.avg()
    
    
    rm = Repository(path_to_repo, single=from_commit).traverse_commits()
    for commit in rm:
        #print(commit.dmm_unit_size)
        #print(commit.dmm_unit_complexity)
        #print(commit.dmm_unit_interfacing)
        
        row['dmm_unit_size'] = commit.dmm_unit_size
        row['dmm_unit_complexity'] = commit.dmm_unit_complexity
        row['dmm_unit_interfacing'] = commit.dmm_unit_interfacing
        
    from pydriller.metrics.process.contributors_count import ContributorsCount
    metric = ContributorsCount(path_to_repo=path_to_repo,
                           from_commit=from_commit,
                           to_commit=to_commit)
    count = metric.count()
    minor = metric.count_minor()
    row['avg_contributor_count'] = mean(list(count.values()))
    row['avg_contributor_count_minor'] = mean(list(minor.values()))
    
    from pydriller.metrics.process.contributors_experience import ContributorsExperience
    metric = ContributorsExperience(path_to_repo=path_to_repo,
                                    from_commit=from_commit,
                                    to_commit=to_commit)
    files = metric.count()
    row['avg_files_contributor'] = mean(list(files.values()))
    
    
    from pydriller.metrics.process.hunks_count import HunksCount
    metric = HunksCount(path_to_repo=path_to_repo,
                                    from_commit=from_commit,
                                    to_commit=to_commit)
    files = metric.count()
    row['avg_hunk'] = mean(list(files.values()))
    
    
    #return (maximum_files_committed_tgt, dmm_unit_size, 
    #        dmm_unit_complexity, dmm_unit_interfacing, 
    #        avg_contributor_count, avg_contributor_count_minor, 
    #        avg_files_contributor,avg_hunk)
    
    return row
    

In [62]:
lol = get_pydriller_process_metrics('https://github.com/Alluxio/alluxio', 'a4dc54f7dc0333da096aa779dbaa79060c90d1ad','1356c0b35fd753d1081665b2c8f6b25da62bf2e8')
print(lol)

(2, 0.0, 1.0, 1.0, 1.0, 0.0, 100.0, 1.5)


In [63]:
main_df = main_df.head()

In [65]:
main_df

,kmean_label,time_taken_mean,time_taken_min,time_taken_max,time_taken_q10,time_taken_q20,time_taken_q25,time_taken_q30,time_taken_q40,time_taken_q50,...,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std,commit_line_changed,refactoring_perc,refactoring_perc_time_taken,cocomoII_time_taken,geneticP_time_taken,softwareMaintenance_time_taken,fuzzy_time_taken,project_link
0,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,2,NaN,8,1.000000,15.00000,4.13952,2.477916,12.48,42.435515,https://github.com/Alluxio/alluxio
1,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,1,NaN,346,0.225434,1.00000,179.03424,107.076009,2229.76,519.589347,https://github.com/Alluxio/alluxio
2,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,2,0.707107,35,1.000000,1.00000,18.11040,10.840124,189.60,113.234791,https://github.com/Alluxio/alluxio
3,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,13,4.018895,194,0.185567,3.71134,100.38336,60.060503,1232.64,353.641141,https://github.com/Alluxio/alluxio
4,5,12.712987,1.0,166.0,1.0,1.0,2.0,2.0,3.0,5.0,...,136,50.585661,1938,0.004128,1.00000,1002.79872,597.273303,12673.28,1634.048307,https://github.com/Alluxio/alluxio


In [74]:
main_df = main_df.apply(get_pydriller_process_metrics, axis=1)

In [75]:
main_df.to_csv('Temp_check.csv', index=False)

In [15]:
# Cocomo formula = 2.94 * kloc * 176

In [ ]:
# GeneticP formula = 1.75992 * kloc - 4.56 * 0.001 * kloc^2

